# CYP TDI 30min assay - 3A4 2C8 2C9 2D6
- The script is designed to process compound*30 (row 1~30) +  PosCtrl*1(row 31) + DMSO*1(row 32)

# Load Packages

In [1]:
pacman::p_load(pacman, dplyr, tidyr, stringr, readxl, writexl, outliers)

# Import Rawdata
### Raw data file format: “.xlsx”
- It is crucial to have only one .xlsx file in the same directory as the script, so it will fetch the correct one;


In [2]:
# Rawdata file name
Name_rawdata_file <- list.files(pattern = "*.xlsx", full.names = T)  
Name_rawdata_file

[1] "./2020-06-22 CYP TDI 30min assay - 3A4 2C8 2C9 2D6 - Experiment.xlsx"

### Rawdata sheet names has to be:
- 3A4_Midazolam_rawdata
- 2C8_Paclitaxel_rawdata
- 2C9_Diclofenac_rawdata
- 2D6_Dextromethorphan_rawdata

### Format of LCMS raw data: "Sample Name", "Analyte Peak Area (counts)" and "IS Peak Area (counts)"
- Column name must be exact match. This is default name used in Analyst software so you shouldn’t have to change it.
- Samples do not have to be in order. Processed data will be merged based on compound list set in experiment file. 

### Sample Name format: "ID"_"(+/-)NADPH"_"Replicate"
- No special character or space is allowed in ID(e.g. A0305554 instead of A-0305554, PosCtrls instead of Pos Ctrls) to avoid the script splitting strings wrongly. 
- If running duplicate sets of 14 compounds, recommend to name compounds like "Cpd01r1 & Cpd01r2", or "Cpd01a and Cpd01b".

In [3]:
Rawdata_3A4 <- read_excel(Name_rawdata_file, sheet = "3A4_Midazolam_rawdata") 
head(Rawdata_3A4)

Sample Name,Analyte Peak Area (counts),IS Peak Area (counts)
A0307356_(+)NADPH_A,557000,2370000
A0307356_(+)NADPH_B,537000,2270000
A0307356_(+)NADPH_C,578000,2270000
A0307358_(+)NADPH_A,623000,2190000
A0307358_(+)NADPH_B,637000,2290000
A0307358_(+)NADPH_C,661000,2270000


In [4]:
Rawdata_2C8 <- read_excel(Name_rawdata_file, sheet = "2C8_Paclitaxel_rawdata")
head(Rawdata_2C8)

Sample Name,Analyte Peak Area (counts),IS Peak Area (counts)
A0307356_(+)NADPH_A,46900,2820000
A0307356_(+)NADPH_B,51700,2640000
A0307356_(+)NADPH_C,54800,2840000
A0307358_(+)NADPH_A,45400,2530000
A0307358_(+)NADPH_B,43500,2730000
A0307358_(+)NADPH_C,47300,2730000


In [5]:
Rawdata_2C9 <- read_excel(Name_rawdata_file, sheet = "2C9_Diclofenac_rawdata")
head(Rawdata_2C9)

Sample Name,Analyte Peak Area (counts),IS Peak Area (counts)
A0307356_(+)NADPH_A,1180000,87500
A0307356_(+)NADPH_B,1180000,87200
A0307356_(+)NADPH_C,1280000,89300
A0307358_(+)NADPH_A,881000,91900
A0307358_(+)NADPH_B,880000,90400
A0307358_(+)NADPH_C,925000,90900


In [6]:
Rawdata_2D6 <- read_excel(Name_rawdata_file, sheet = "2D6_Dextromethorphan_rawdata")
head(Rawdata_2D6)

Sample Name,Analyte Peak Area (counts),IS Peak Area (counts)
A0307356_(+)NADPH_A,168000,1840000
A0307356_(+)NADPH_B,168000,1770000
A0307356_(+)NADPH_C,177000,1770000
A0307358_(+)NADPH_A,183000,1730000
A0307358_(+)NADPH_B,190000,1740000
A0307358_(+)NADPH_C,201000,1750000


### Compound list is used in the script, please set it accordingly.
- Sheet name has to be "Compound List".
- No special character or space in Arcus ID column (e.g. A0305554 instead of A-0305554, PosCtrls instead of Pos Ctrls)
- The script will cleanup dataframe to contain only row 8 to 39, column 2 to 5 in the template file.

In [7]:
Df_Cpd_List <- read_excel(Name_rawdata_file, sheet = "Compound List")
Df_Cpd_List <- Df_Cpd_List[7:38, 2:5]
colnames(Df_Cpd_List) <- c("ID", "Project", "Batch", "Concentration (uM)")
Df_Cpd_List

New names:
* `` -> ...1
* `` -> ...4
* `` -> ...5
* `` -> ...6
* `` -> ...8
* ...


ID,Project,Batch,Concentration (uM)
A0307356,NA,1,10
A0307358,NA,1,10
A0307375,NA,1,10
A0307377,NA,1,10
A0307386,NA,1,10
A0307387,NA,1,10
A0307388,NA,1,10
A0307389,NA,1,10
A0307391,NA,1,10
A0307396,NA,1,10


# Define Variables
- Make sure the values are set to what you want for data processing.

In [8]:
Val_replicate <- 3                                          # 3 for triplicates
Val_pct_CV_cutoff <- 20                                     # Cut-off value of %CV  
Val_inc_time = 30                                           # Pre-incubation time

# Prepare Df_Cpd_List to match summary file format 

In [9]:
Df_Cpd_List <- mutate(Df_Cpd_List,
                      "Structure" = "",
                      "Pre-inc time (min)" = Val_inc_time)
Df_Cpd_List <- Df_Cpd_List[c(32, 31, 1:30), c(1,3,2,5,4,6)]
head(Df_Cpd_List)

ID,Batch,Project,Structure,Concentration (uM),Pre-inc time (min)
DMSO,NA,NA,,NA,30
ABT,NA,PosCtrl,,500,30
A0307356,1,NA,,10,30
A0307358,1,NA,,10,30
A0307375,1,NA,,10,30
A0307377,1,NA,,10,30


# Remove MeOH Wash Samples

In [10]:
Rawdata_3A4 <- Rawdata_3A4[!grepl("MeOH", Rawdata_3A4$`Sample Name`), ]                     
Rawdata_2C8 <- Rawdata_2C8[!grepl("MeOH", Rawdata_2C8$`Sample Name`), ]                     
Rawdata_2C9 <- Rawdata_2C9[!grepl("MeOH", Rawdata_2C9$`Sample Name`), ]     
Rawdata_2D6 <- Rawdata_2D6[!grepl("MeOH", Rawdata_2D6$`Sample Name`), ]  

# Split strings in sample name

In [11]:
Rawdata_3A4 <- Rawdata_3A4 %>% separate(`Sample Name`, c("ID", "NADPH", "Replicate"), "_")
Rawdata_2C8 <- Rawdata_2C8 %>% separate(`Sample Name`, c("ID", "NADPH", "Replicate"), "_")
Rawdata_2C9 <- Rawdata_2C9 %>% separate(`Sample Name`, c("ID", "NADPH", "Replicate"), "_")
Rawdata_2D6 <- Rawdata_2D6 %>% separate(`Sample Name`, c("ID", "NADPH", "Replicate"), "_")

head(Rawdata_3A4)

ID,NADPH,Replicate,Analyte Peak Area (counts),IS Peak Area (counts)
A0307356,(+)NADPH,A,557000,2370000
A0307356,(+)NADPH,B,537000,2270000
A0307356,(+)NADPH,C,578000,2270000
A0307358,(+)NADPH,A,623000,2190000
A0307358,(+)NADPH,B,637000,2290000
A0307358,(+)NADPH,C,661000,2270000


# Calculate PAR (Analyte to ISTD Peak Area Ratio)

In [12]:
Rawdata_3A4 <- mutate(Rawdata_3A4, PAR = Rawdata_3A4$`Analyte Peak Area (counts)`/Rawdata_3A4$`IS Peak Area (counts)`)
Rawdata_2C8 <- mutate(Rawdata_2C8, PAR = Rawdata_2C8$`Analyte Peak Area (counts)`/Rawdata_2C8$`IS Peak Area (counts)`)
Rawdata_2C9 <- mutate(Rawdata_2C9, PAR = Rawdata_2C9$`Analyte Peak Area (counts)`/Rawdata_2C9$`IS Peak Area (counts)`)
Rawdata_2D6 <- mutate(Rawdata_2D6, PAR = Rawdata_2D6$`Analyte Peak Area (counts)`/Rawdata_2D6$`IS Peak Area (counts)`)

head(Rawdata_3A4)

ID,NADPH,Replicate,Analyte Peak Area (counts),IS Peak Area (counts),PAR
A0307356,(+)NADPH,A,557000,2370000,0.2350211
A0307356,(+)NADPH,B,537000,2270000,0.2365639
A0307356,(+)NADPH,C,578000,2270000,0.2546256
A0307358,(+)NADPH,A,623000,2190000,0.2844749
A0307358,(+)NADPH,B,637000,2290000,0.2781659
A0307358,(+)NADPH,C,661000,2270000,0.2911894


# Split (+) and (-) NADPH Samples into Different Dataframes

In [13]:
Rawdata_3A4_plus <- filter(Rawdata_3A4, Rawdata_3A4$NADPH == "(+)NADPH")
Rawdata_3A4_minus <- filter(Rawdata_3A4, Rawdata_3A4$NADPH == "(-)NADPH")
Rawdata_2C8_plus <- filter(Rawdata_2C8, Rawdata_2C8$NADPH == "(+)NADPH")
Rawdata_2C8_minus <- filter(Rawdata_2C8, Rawdata_2C8$NADPH == "(-)NADPH")
Rawdata_2C9_plus <- filter(Rawdata_2C9, Rawdata_2C9$NADPH == "(+)NADPH")
Rawdata_2C9_minus <- filter(Rawdata_2C9, Rawdata_2C9$NADPH == "(-)NADPH")
Rawdata_2D6_plus <- filter(Rawdata_2D6, Rawdata_2D6$NADPH == "(+)NADPH")
Rawdata_2D6_minus <- filter(Rawdata_2D6, Rawdata_2D6$NADPH == "(-)NADPH")

head(Rawdata_3A4_plus)
head(Rawdata_3A4_minus)

ID,NADPH,Replicate,Analyte Peak Area (counts),IS Peak Area (counts),PAR
A0307356,(+)NADPH,A,557000,2370000,0.2350211
A0307356,(+)NADPH,B,537000,2270000,0.2365639
A0307356,(+)NADPH,C,578000,2270000,0.2546256
A0307358,(+)NADPH,A,623000,2190000,0.2844749
A0307358,(+)NADPH,B,637000,2290000,0.2781659
A0307358,(+)NADPH,C,661000,2270000,0.2911894


ID,NADPH,Replicate,Analyte Peak Area (counts),IS Peak Area (counts),PAR
A0307356,(-)NADPH,A,801000,2700000,0.2966667
A0307356,(-)NADPH,B,774000,2580000,0.3000000
A0307356,(-)NADPH,C,820000,2830000,0.2897527
A0307358,(-)NADPH,A,956000,2630000,0.3634981
A0307358,(-)NADPH,B,919000,2630000,0.3494297
A0307358,(-)NADPH,C,941000,2610000,0.3605364


# Create functions 
### mean.rm.o(Data)
- Calculate avg without the value most differing from mean

In [14]:
mean.rm.o <- function(Data) { zval_1 <- rm.outlier(Data)
                              zval_2 <- mean(zval_1, na.rm = TRUE)
                              return(zval_2)} 

print(Rawdata_3A4_plus$PAR[4:6])         # print PAR values of row 4 to 6 in dataframe "Rawdata_3A4_plus"
mean(Rawdata_3A4_plus$PAR[4:6])          # average PAR of all three 
mean.rm.o(Rawdata_3A4_plus$PAR[4:6])     # average PAR without the one most differing from mean

[1] 0.2844749 0.2781659 0.2911894


[1] 0.2846101

[1] 0.2813204

### pct_CT(Data)
- Calculate % CV

In [15]:
pct_CV <- function(Data) { zval_1 <- mean(Data, na.rm = TRUE)
                           zval_2 <- sd(Data, na.rm = TRUE)
                           zval_3 <- zval_2/zval_1*100
                           zval_4 <- round(zval_3, 1)
                           return(zval_4)} 

print(Rawdata_3A4_plus$PAR[4:6])         # print PAR values of row 4 to 6 in dataframe "Rawdata_3A4_plus"
pct_CV(Rawdata_3A4_plus$PAR[4:6])        # calculate % CV

[1] 0.2844749 0.2781659 0.2911894


[1] 2.3

### pct_CV.rm.o(Data) 
- Calculate % CV without the value most differing from mean

In [16]:
pct_CV.rm.o <- function(Data) { zval_1 <- rm.outlier(Data)
                                zval_2 <- mean(zval_1, na.rm = TRUE)
                                zval_3 <- sd(zval_1, na.rm = TRUE)
                                zval_4 <- zval_3/zval_2*100
                                zval_5 <- round(zval_4, 1)
                                return(zval_5)} 

print(Rawdata_3A4_plus$PAR[4:6])         # print PAR values of row 4 to 6 in dataframe "Rawdata_3A4_plus"
pct_CV.rm.o(Rawdata_3A4_plus$PAR[4:6])   # calculate % CV without an outlier most differing from mean

[1] 0.2844749 0.2781659 0.2911894


[1] 1.6

# Calculate % activity remaining for 3A4 (+) NADPH samples
- Criteria to exclude an outlier in triplicate is set as %CV > 20 in this script (you can change cutoff value if needed); 
- Script will not return any value if %CV is still > cutoff value after exclude one outlier.

In [17]:
# Calculate average and %CV with or without an outlier
Df_CYP3A4_plus <- Rawdata_3A4_plus %>%                                                                   
                  group_by(ID) %>%
                  summarize(PAR_avg = mean(PAR, na.rm = TRUE),
                            Pct_CV = pct_CV(PAR),
                            PAR_avg_rm_o = mean.rm.o(PAR),
                            Pct_CV_rm_o = pct_CV.rm.o(PAR))

# Find the row number containing "DMSO" in sample name
Val_row_num_DMSO <- str_which(Df_CYP3A4_plus$ID, "DMSO")                                         

# If %CV > cutoff, use the average without outlier; return"NA" if %CV is still > cutoff after remove one outlier 
Df_CYP3A4_plus <- Df_CYP3A4_plus %>%                                                                           
    mutate("PAR_avg (+)" = ifelse(Df_CYP3A4_plus$Pct_CV < Val_pct_CV_cutoff, Df_CYP3A4_plus$PAR_avg, 
                              ifelse(Df_CYP3A4_plus$Pct_CV_rm_o < Val_pct_CV_cutoff, Df_CYP3A4_plus$PAR_avg_rm_o, NA)
                                 )
          )
Df_CYP3A4_plus <- Df_CYP3A4_plus %>% 
    mutate("PAR_avg_DMSO (+)" = Df_CYP3A4_plus$`PAR_avg (+)`[Val_row_num_DMSO])

Df_CYP3A4_plus <- Df_CYP3A4_plus %>% 
    mutate("3A4 % Activity remaining (+)" = round(Df_CYP3A4_plus$`PAR_avg (+)`/Df_CYP3A4_plus$`PAR_avg_DMSO (+)`*100, 1))

head(Df_CYP3A4_plus)

ID,PAR_avg,Pct_CV,PAR_avg_rm_o,Pct_CV_rm_o,PAR_avg (+),PAR_avg_DMSO (+),3A4 % Activity remaining (+)
A0307356,0.24207017,4.5,0.23579249,0.5,0.24207017,0.36553,66.2
A0307358,0.28461008,2.3,0.28132041,1.6,0.28461008,0.36553,77.9
A0307375,0.17915329,7.5,0.18669531,2.1,0.17915329,0.36553,49.0
A0307377,0.04453165,18.5,0.04902224,7.8,0.04453165,0.36553,12.2
A0307386,0.11322466,20.2,0.10020136,5.0,0.10020136,0.36553,27.4
A0307387,0.14287037,2.8,0.14513889,0.7,0.14287037,0.36553,39.1


# Calculate % activity remaining for 3A4 (-) NADPH samples
- Criteria to exclude an outlier in triplicate is set as %CV > 20 in this script (you can change cutoff value if needed); 
- Script will not return any value if %CV is still > cutoff value after exclude one outlier.

In [18]:
# Calculate average and %CV with or without an outlier
Df_CYP3A4_minus <- Rawdata_3A4_minus %>%                                                                   
                   group_by(ID) %>%
                   summarize(PAR_avg = mean(PAR, na.rm = TRUE),
                             Pct_CV = pct_CV(PAR),
                             PAR_avg_rm_o = mean.rm.o(PAR),
                             Pct_CV_rm_o = pct_CV.rm.o(PAR))
    
# find the row number containing "DMSO" in sample name
Val_row_num_DMSO <- str_which(Df_CYP3A4_minus$ID, "DMSO")                                         

# If %CV > cutoff, use the average without outlier; return"NA" if %CV is still > cutoff after remove one outlier 
Df_CYP3A4_minus <- Df_CYP3A4_minus %>%                                                                           
    mutate("PAR_avg (-)" = ifelse(Df_CYP3A4_minus$Pct_CV < Val_pct_CV_cutoff, Df_CYP3A4_minus$PAR_avg, 
                            ifelse(Df_CYP3A4_minus$Pct_CV_rm_o < Val_pct_CV_cutoff, Df_CYP3A4_minus$PAR_avg_rm_o, NA)
                                 )
          )

Df_CYP3A4_minus <- Df_CYP3A4_minus %>% 
    mutate("PAR_avg_DMSO (-)" = Df_CYP3A4_minus$`PAR_avg (-)`[Val_row_num_DMSO])

Df_CYP3A4_minus <- Df_CYP3A4_minus %>% 
    mutate("3A4 % Activity remaining (-)" = round(Df_CYP3A4_minus$`PAR_avg (-)`/Df_CYP3A4_minus$`PAR_avg_DMSO (-)`*100, 1))

head(Df_CYP3A4_minus)

ID,PAR_avg,Pct_CV,PAR_avg_rm_o,Pct_CV_rm_o,PAR_avg (-),PAR_avg_DMSO (-),3A4 % Activity remaining (-)
A0307356,0.29547311,1.8,0.29833333,0.8,0.29547311,0.4024438,73.4
A0307358,0.35782139,2.1,0.36201725,0.6,0.35782139,0.4024438,88.9
A0307375,0.23096760,5.2,0.23764990,1.8,0.23096760,0.4024438,57.4
A0307377,0.08995185,7.5,0.08609509,1.1,0.08995185,0.4024438,22.4
A0307386,0.22312507,1.1,0.22442641,0.4,0.22312507,0.4024438,55.4
A0307387,0.32555893,1.4,0.32804428,0.5,0.32555893,0.4024438,80.9


# Calculate % activity remaining for 2C8 (+) NADPH samples
- Criteria to exclude an outlier in triplicate is set as %CV > 20 in this script (you can change cutoff value if needed); 
- Script will not return any value if %CV is still > cutoff value after exclude one outlier.

In [19]:
# Calculate average and %CV with or without an outlier
Df_CYP2C8_plus <- Rawdata_2C8_plus %>%                                                                   
                  group_by(ID) %>%
                  summarize(PAR_avg = mean(PAR, na.rm = TRUE),
                            Pct_CV = pct_CV(PAR),
                            PAR_avg_rm_o = mean.rm.o(PAR),
                            Pct_CV_rm_o = pct_CV.rm.o(PAR))

# find the row number containing "DMSO" in sample name
Val_row_num_DMSO <- str_which(Df_CYP2C8_plus$ID, "DMSO")                                          

# If %CV > cutoff, use the average without outlier; return"NA" if %CV is still > cutoff after remove one outlier 
Df_CYP2C8_plus <- Df_CYP2C8_plus %>%                                                                           
    mutate("PAR_avg (+)" = ifelse(Df_CYP2C8_plus$Pct_CV < Val_pct_CV_cutoff, Df_CYP2C8_plus$PAR_avg, 
                              ifelse(Df_CYP2C8_plus$Pct_CV_rm_o < Val_pct_CV_cutoff, Df_CYP2C8_plus$PAR_avg_rm_o, NA)
                                 )
          )

Df_CYP2C8_plus <- Df_CYP2C8_plus %>% 
    mutate("PAR_avg_DMSO (+)" = Df_CYP2C8_plus$`PAR_avg (+)`[Val_row_num_DMSO])

Df_CYP2C8_plus <- Df_CYP2C8_plus %>% 
    mutate("2C8 % Activity remaining (+)" = round(Df_CYP2C8_plus$`PAR_avg (+)`/Df_CYP2C8_plus$`PAR_avg_DMSO (+)`*100, 1))

head(Df_CYP2C8_plus)

ID,PAR_avg,Pct_CV,PAR_avg_rm_o,Pct_CV_rm_o,PAR_avg (+),PAR_avg_DMSO (+),2C8 % Activity remaining (+)
A0307356,0.018503438,8.8,0.019439554,1.0,0.018503438,0.02088912,88.6
A0307358,0.017068246,6.0,0.017635336,2.5,0.017068246,0.02088912,81.7
A0307375,0.012048023,5.6,0.011675482,2.6,0.012048023,0.02088912,57.7
A0307377,0.008249985,4.9,0.008455118,3.2,0.008249985,0.02088912,39.5
A0307386,0.009744530,6.9,0.009362249,1.7,0.009744530,0.02088912,46.6
A0307387,0.012765002,3.6,0.013027373,0.9,0.012765002,0.02088912,61.1


# Calculate % activity remaining for 2C8 (-) NADPH samples
- Criteria to exclude an outlier in triplicate is set as %CV > 20 in this script (you can change cutoff value if needed); 
- Script will not return any value if %CV is still > cutoff value after exclude one outlier.

In [20]:
# Calculate average and %CV with or without an outlier
Df_CYP2C8_minus <- Rawdata_2C8_minus %>%                                                                   
  group_by(ID) %>%
  summarize(PAR_avg = mean(PAR, na.rm = TRUE),
            Pct_CV = pct_CV(PAR),
            PAR_avg_rm_o = mean.rm.o(PAR),
            Pct_CV_rm_o = pct_CV.rm.o(PAR))

# find the row number containing "DMSO" in sample name
Val_row_num_DMSO <- str_which(Df_CYP2C8_minus$ID, "DMSO")                                         

# If %CV > cutoff, use the average without outlier; return"NA" if %CV is still > cutoff after remove one outlier 
Df_CYP2C8_minus <- Df_CYP2C8_minus %>%                                                                           
    mutate("PAR_avg (-)" = ifelse(Df_CYP2C8_minus$Pct_CV < Val_pct_CV_cutoff, Df_CYP2C8_minus$PAR_avg, 
                              ifelse(Df_CYP2C8_minus$Pct_CV_rm_o < Val_pct_CV_cutoff, Df_CYP2C8_minus$PAR_avg_rm_o, NA)
                                 )
          )

Df_CYP2C8_minus <- Df_CYP2C8_minus %>% 
    mutate("PAR_avg_DMSO (-)" = Df_CYP2C8_minus$`PAR_avg (-)`[Val_row_num_DMSO])

Df_CYP2C8_minus <- Df_CYP2C8_minus %>% 
    mutate("2C8 % Activity remaining (-)" = round(Df_CYP2C8_minus$`PAR_avg (-)`/Df_CYP2C8_minus$`PAR_avg_DMSO (-)`*100, 1))

head(Df_CYP2C8_minus)

ID,PAR_avg,Pct_CV,PAR_avg_rm_o,Pct_CV_rm_o,PAR_avg (-),PAR_avg_DMSO (-),2C8 % Activity remaining (-)
A0307356,0.02394984,5.2,0.02462445,2.3,0.02394984,0.02335181,102.6
A0307358,0.02109719,7.8,0.02021486,4.2,0.02109719,0.02335181,90.3
A0307375,0.01732335,4.5,0.01777190,0.1,0.01732335,0.02335181,74.2
A0307377,0.01214074,4.9,0.01245790,2.5,0.01214074,0.02335181,52.0
A0307386,0.01460331,3.3,0.01487748,0.3,0.01460331,0.02335181,62.5
A0307387,0.01949653,6.6,0.01877401,2.2,0.01949653,0.02335181,83.5


# Calculate % activity remaining for 2C9 (+) NADPH samples
- Criteria to exclude an outlier in triplicate is set as %CV > 20 in this script (you can change cutoff value if needed); 
- Script will not return any value if %CV is still > cutoff value after exclude one outlier.

In [21]:
# Calculate average and %CV with or without an outlier
Df_CYP2C9_plus <- Rawdata_2C9_plus %>%                                                                   
                  group_by(ID) %>%
                  summarize(PAR_avg = mean(PAR, na.rm = TRUE),
                            Pct_CV = pct_CV(PAR),
                            PAR_avg_rm_o = mean.rm.o(PAR),
                            Pct_CV_rm_o = pct_CV.rm.o(PAR))

# find the row number containing "DMSO" in sample name
Val_row_num_DMSO <- str_which(Df_CYP2C9_plus$ID, "DMSO")                                          

# If %CV > cutoff, use the average without outlier; return"NA" if %CV is still > cutoff after remove one outlier 
Df_CYP2C9_plus <- Df_CYP2C9_plus %>%                                                                           
    mutate("PAR_avg (+)" = ifelse(Df_CYP2C9_plus$Pct_CV < Val_pct_CV_cutoff, Df_CYP2C9_plus$PAR_avg, 
                              ifelse(Df_CYP2C9_plus$Pct_CV_rm_o < Val_pct_CV_cutoff, Df_CYP2C9_plus$PAR_avg_rm_o, NA)
                                 )
          )

Df_CYP2C9_plus <- Df_CYP2C9_plus %>% 
    mutate("PAR_avg_DMSO (+)" = Df_CYP2C9_plus$`PAR_avg (+)`[Val_row_num_DMSO])

Df_CYP2C9_plus <- Df_CYP2C9_plus %>% 
    mutate("2C9 % Activity remaining (+)" = round(Df_CYP2C9_plus$`PAR_avg (+)`/Df_CYP2C9_plus$`PAR_avg_DMSO (+)`*100, 1))

head(Df_CYP2C9_plus)

ID,PAR_avg,Pct_CV,PAR_avg_rm_o,Pct_CV_rm_o,PAR_avg (+),PAR_avg_DMSO (+),2C9 % Activity remaining (+)
A0307356,13.783844,3.5,13.508912,0.2,13.783844,17.4633,78.9
A0307358,9.832346,3.1,9.660510,1.1,9.832346,17.4633,56.3
A0307375,13.019338,3.6,12.781219,2.3,13.019338,17.4633,74.6
A0307377,9.814369,6.8,9.463312,4.1,9.814369,17.4633,56.2
A0307386,10.726333,16.1,9.793203,8.8,10.726333,17.4633,61.4
A0307387,13.430065,1.7,13.303533,0.7,13.430065,17.4633,76.9


# Calculate % activity remaining for 2C9 (-) NADPH samples
- Criteria to exclude an outlier in triplicate is set as %CV > 20 in this script (you can change cutoff value if needed); 
- Script will not return any value if %CV is still > cutoff value after exclude one outlier.

In [22]:
# Calculate average and %CV with or without an outlier
Df_CYP2C9_minus <- Rawdata_2C9_minus %>%                                                                   
                   group_by(ID) %>%
                   summarize(PAR_avg = mean(PAR, na.rm = TRUE),
                             Pct_CV = pct_CV(PAR),
                             PAR_avg_rm_o = mean.rm.o(PAR),
                             Pct_CV_rm_o = pct_CV.rm.o(PAR))

# find the row number containing "DMSO" in sample name
Val_row_num_DMSO <- str_which(Df_CYP2C9_minus$ID, "DMSO")                                        

# If %CV > cutoff, use the average without outlier; return"NA" if %CV is still > cutoff after remove one outlier 
Df_CYP2C9_minus <- Df_CYP2C9_minus %>%                                                                           
    mutate("PAR_avg (-)" = ifelse(Df_CYP2C9_minus$Pct_CV < Val_pct_CV_cutoff, Df_CYP2C9_minus$PAR_avg, 
                              ifelse(Df_CYP2C9_minus$Pct_CV_rm_o < Val_pct_CV_cutoff, Df_CYP2C9_minus$PAR_avg_rm_o, NA)
                                 )
          )

Df_CYP2C9_minus <- Df_CYP2C9_minus %>% 
    mutate("PAR_avg_DMSO (-)" = Df_CYP2C9_minus$`PAR_avg (-)`[Val_row_num_DMSO])

Df_CYP2C9_minus <- Df_CYP2C9_minus %>% 
    mutate("2C9 % Activity remaining (-)" = round(Df_CYP2C9_minus$`PAR_avg (-)`/Df_CYP2C9_minus$`PAR_avg_DMSO (-)`*100, 1))

head(Df_CYP2C9_minus)

ID,PAR_avg,Pct_CV,PAR_avg_rm_o,Pct_CV_rm_o,PAR_avg (-),PAR_avg_DMSO (-),2C9 % Activity remaining (-)
A0307356,15.73439,3.2,15.47075,1.8,15.73439,17.28127,91.0
A0307358,12.34222,6.2,11.89874,0.8,12.34222,17.28127,71.4
A0307375,16.02312,4.3,15.62298,0.3,16.02312,17.28127,92.7
A0307377,12.36245,1.6,12.47572,0.0,12.36245,17.28127,71.5
A0307386,14.81482,3.3,14.54897,1.6,14.81482,17.28127,85.7
A0307387,17.08028,5.3,16.56102,0.1,17.08028,17.28127,98.8


# Calculate % activity remaining for 2D6 (+) NADPH samples
- Criteria to exclude an outlier in triplicate is set as %CV > 20 in this script (you can change cutoff value if needed); 
- Script will not return any value if %CV is still > cutoff value after exclude one outlier.

In [23]:
# Calculate average and %CV with or without an outlier
Df_CYP2D6_plus <- Rawdata_2D6_plus %>%                                                                   
                  group_by(ID) %>%
                  summarize(PAR_avg = mean(PAR, na.rm = TRUE),
                            Pct_CV = pct_CV(PAR),
                            PAR_avg_rm_o = mean.rm.o(PAR),
                            Pct_CV_rm_o = pct_CV.rm.o(PAR))

# find the row number containing "DMSO" in sample name
Val_row_num_DMSO <- str_which(Df_CYP2D6_plus$ID, "DMSO")                                         

# If %CV > cutoff, use the average without outlier; return"NA" if %CV is still > cutoff after remove one outlier 
Df_CYP2D6_plus <- Df_CYP2D6_plus %>%                                                                           
    mutate("PAR_avg (+)" = ifelse(Df_CYP2D6_plus$Pct_CV < Val_pct_CV_cutoff, Df_CYP2D6_plus$PAR_avg, 
                              ifelse(Df_CYP2D6_plus$Pct_CV_rm_o < Val_pct_CV_cutoff, Df_CYP2D6_plus$PAR_avg_rm_o, NA)
                                 )
          )

Df_CYP2D6_plus <- Df_CYP2D6_plus %>% 
    mutate("PAR_avg_DMSO (+)" = Df_CYP2D6_plus$`PAR_avg (+)`[Val_row_num_DMSO])

Df_CYP2D6_plus <- Df_CYP2D6_plus %>% 
    mutate("2D6 % Activity remaining (+)" = round(Df_CYP2D6_plus$`PAR_avg (+)`/Df_CYP2D6_plus$`PAR_avg_DMSO (+)`*100, 1))

head(Df_CYP2D6_plus)

ID,PAR_avg,Pct_CV,PAR_avg_rm_o,Pct_CV_rm_o,PAR_avg (+),PAR_avg_DMSO (+),2D6 % Activity remaining (+)
A0307356,0.09540653,4.6,0.09310980,2.7,0.09540653,0.1339678,71.2
A0307358,0.10994430,4.2,0.10748787,2.2,0.10994430,0.1339678,82.1
A0307375,0.08580762,4.7,0.08354097,1.7,0.08580762,0.1339678,64.1
A0307377,0.09742425,5.3,0.09443183,0.3,0.09742425,0.1339678,72.7
A0307386,0.10157407,12.2,0.10861111,3.3,0.10157407,0.1339678,75.8
A0307387,0.11163025,3.6,0.11383427,1.7,0.11163025,0.1339678,83.3


# Calculate % activity remaining for 2D6 (-) NADPH samples
- Criteria to exclude an outlier in triplicate is set as %CV > 20 in this script (you can change cutoff value if needed); 
- Script will not return any value if %CV is still > cutoff value after exclude one outlier.

In [24]:
Df_CYP2D6_minus <- Rawdata_2D6_minus %>%                                                                   
                   group_by(ID) %>%
                   summarize(PAR_avg = mean(PAR, na.rm = TRUE),
                             Pct_CV = pct_CV(PAR),
                             PAR_avg_rm_o = mean.rm.o(PAR),
                             Pct_CV_rm_o = pct_CV.rm.o(PAR))

# find the row number containing "DMSO" in sample name
Val_row_num_DMSO <- str_which(Df_CYP2D6_minus$ID, "DMSO")                                         

# If %CV > cutoff, use the average without outlier; return"NA" if %CV is still > cutoff after remove one outlier 
Df_CYP2D6_minus <- Df_CYP2D6_minus %>%                                                                           
    mutate("PAR_avg (-)" = ifelse(Df_CYP2D6_minus$Pct_CV < Val_pct_CV_cutoff, Df_CYP2D6_minus$PAR_avg, 
                                ifelse(Df_CYP2D6_minus$Pct_CV_rm_o < Val_pct_CV_cutoff, Df_CYP2D6_minus$PAR_avg_rm_o, NA)
                               )
          )

Df_CYP2D6_minus <- Df_CYP2D6_minus %>% 
    mutate("PAR_avg_DMSO (-)" = Df_CYP2D6_minus$`PAR_avg (-)`[Val_row_num_DMSO])

Df_CYP2D6_minus <- Df_CYP2D6_minus %>% 
    mutate("2D6 % Activity remaining (-)" = round(Df_CYP2D6_minus$`PAR_avg (-)`/Df_CYP2D6_minus$`PAR_avg_DMSO (-)`*100, 1))

head(Df_CYP2D6_minus)

ID,PAR_avg,Pct_CV,PAR_avg_rm_o,Pct_CV_rm_o,PAR_avg (-),PAR_avg_DMSO (-),2D6 % Activity remaining (-)
A0307356,0.09798360,2.2,0.09922540,0.4,0.09798360,0.1478038,66.3
A0307358,0.13876090,2.1,0.14043301,0.7,0.13876090,0.1478038,93.9
A0307375,0.09682417,4.1,0.09892046,2.3,0.09682417,0.1478038,65.5
A0307377,0.12487232,2.1,0.12640422,0.3,0.12487232,0.1478038,84.5
A0307386,0.15054395,1.4,0.14945723,1.0,0.15054395,0.1478038,101.9
A0307387,0.15000714,2.3,0.15181483,1.4,0.15000714,0.1478038,101.5


# Data compilation
## Processed data

In [25]:
Processed_data <- do.call("cbind", list(Df_CYP3A4_plus[,c(1,8)], Df_CYP3A4_minus[8], Df_CYP2C8_plus[8], Df_CYP2C8_minus[8], 
                                       Df_CYP2C9_plus[8], Df_CYP2C9_minus[8], Df_CYP2D6_plus[8], Df_CYP2D6_minus[8]))

Processed_data <- Processed_data %>% 
    mutate("3A4 % Activity Loss" = `3A4 % Activity remaining (-)` - `3A4 % Activity remaining (+)`,
           "2C8 % Activity Loss" = `2C8 % Activity remaining (-)` - `2C8 % Activity remaining (+)`,
           "2C9 % Activity Loss" = `2C9 % Activity remaining (-)` - `2C9 % Activity remaining (+)`,
           "2D6 % Activity Loss" = `2D6 % Activity remaining (-)` - `2D6 % Activity remaining (+)`)

Processed_data$`3A4 % Activity Loss` <- pmax(Processed_data$`3A4 % Activity Loss`,0)
Processed_data$`2C8 % Activity Loss` <- pmax(Processed_data$`2C8 % Activity Loss`,0)
Processed_data$`2C9 % Activity Loss` <- pmax(Processed_data$`2C9 % Activity Loss`,0)
Processed_data$`2D6 % Activity Loss` <- pmax(Processed_data$`2D6 % Activity Loss`,0)

Processed_data <- Processed_data[, c(1:3, 10, 4:5, 11, 6:7, 12, 8:9, 13)]

head(Processed_data)

ID,3A4 % Activity remaining (+),3A4 % Activity remaining (-),3A4 % Activity Loss,2C8 % Activity remaining (+),2C8 % Activity remaining (-),2C8 % Activity Loss,2C9 % Activity remaining (+),2C9 % Activity remaining (-),2C9 % Activity Loss,2D6 % Activity remaining (+),2D6 % Activity remaining (-),2D6 % Activity Loss
A0307356,66.2,73.4,7.2,88.6,102.6,14.0,78.9,91.0,12.1,71.2,66.3,0.0
A0307358,77.9,88.9,11.0,81.7,90.3,8.6,56.3,71.4,15.1,82.1,93.9,11.8
A0307375,49.0,57.4,8.4,57.7,74.2,16.5,74.6,92.7,18.1,64.1,65.5,1.4
A0307377,12.2,22.4,10.2,39.5,52.0,12.5,56.2,71.5,15.3,72.7,84.5,11.8
A0307386,27.4,55.4,28.0,46.6,62.5,15.9,61.4,85.7,24.3,75.8,101.9,26.1
A0307387,39.1,80.9,41.8,61.1,83.5,22.4,76.9,98.8,21.9,83.3,101.5,18.2


## Summary file

In [26]:
Data_Summary <- left_join(Df_Cpd_List, Processed_data, "ID") 
head(Data_Summary)

ID,Batch,Project,Structure,Concentration (uM),Pre-inc time (min),3A4 % Activity remaining (+),3A4 % Activity remaining (-),3A4 % Activity Loss,2C8 % Activity remaining (+),2C8 % Activity remaining (-),2C8 % Activity Loss,2C9 % Activity remaining (+),2C9 % Activity remaining (-),2C9 % Activity Loss,2D6 % Activity remaining (+),2D6 % Activity remaining (-),2D6 % Activity Loss
DMSO,NA,NA,,NA,30,100.0,100.0,0.0,100.0,100.0,0.0,100.0,100.0,0.0,100.0,100.0,0.0
ABT,NA,PosCtrl,,500,30,0.8,20.5,19.7,7.0,83.2,76.2,54.6,111.1,56.5,14.5,113.4,98.9
A0307356,1,NA,,10,30,66.2,73.4,7.2,88.6,102.6,14.0,78.9,91.0,12.1,71.2,66.3,0.0
A0307358,1,NA,,10,30,77.9,88.9,11.0,81.7,90.3,8.6,56.3,71.4,15.1,82.1,93.9,11.8
A0307375,1,NA,,10,30,49.0,57.4,8.4,57.7,74.2,16.5,74.6,92.7,18.1,64.1,65.5,1.4
A0307377,1,NA,,10,30,12.2,22.4,10.2,39.5,52.0,12.5,56.2,71.5,15.3,72.7,84.5,11.8


## CDD upload file

In [27]:
Num_cpd <- nrow(Df_CYP3A4_plus)
CDD_upload_3A4 <- left_join(Df_Cpd_List[3:Num_cpd, c(1:2,4,6)], Processed_data[, c(1:4)], "ID")
CDD_upload_2C8 <- left_join(Df_Cpd_List[3:Num_cpd, c(1:2,4,6)], Processed_data[, c(1,5:7)], "ID")
CDD_upload_2C9 <- left_join(Df_Cpd_List[3:Num_cpd, c(1:2,4,6)], Processed_data[, c(1,8:10)], "ID")
CDD_upload_2D6 <- left_join(Df_Cpd_List[3:Num_cpd, c(1:2,4,6)], Processed_data[, c(1,11:13)], "ID")

colnames(CDD_upload_3A4) <- c("Molecule_Name", "Batch", "Concentration (µM)", "Pre_Incubation_Time (min)", "Pct_Plus_NADPH", "Pct_Minus_NADPH", "Pct_Activity_Loss")
colnames(CDD_upload_2C8) <- c("Molecule_Name", "Batch", "Concentration (µM)", "Pre_Incubation_Time (min)", "Pct_Plus_NADPH", "Pct_Minus_NADPH", "Pct_Activity_Loss")
colnames(CDD_upload_2C9) <- c("Molecule_Name", "Batch", "Concentration (µM)", "Pre_Incubation_Time (min)", "Pct_Plus_NADPH", "Pct_Minus_NADPH", "Pct_Activity_Loss")
colnames(CDD_upload_2D6) <- c("Molecule_Name", "Batch", "Concentration (µM)", "Pre_Incubation_Time (min)", "Pct_Plus_NADPH", "Pct_Minus_NADPH", "Pct_Activity_Loss")

CDD_upload_all <- bind_rows(CDD_upload_3A4, CDD_upload_2C8)
CDD_upload_all <- bind_rows(CDD_upload_all, CDD_upload_2C9)
CDD_upload_all <- bind_rows(CDD_upload_all, CDD_upload_2D6)
CDD_upload_all <- mutate(CDD_upload_all, 
                         "Enzyme (eg 3A4, 2D6, 1A2)" = rep(c("3A4", "2C8", "2C9", "2D6"), each = Num_cpd - 2),
                         "Comment" = "")
CDD_upload_all <- CDD_upload_all[ ,c(1:2, 8, 3:7, 9)]

head(CDD_upload_all)

Molecule_Name,Batch,"Enzyme (eg 3A4, 2D6, 1A2)",Concentration (µM),Pre_Incubation_Time (min),Pct_Plus_NADPH,Pct_Minus_NADPH,Pct_Activity_Loss,Comment
A0307356,1,3A4,,30,66.2,73.4,7.2,
A0307358,1,3A4,,30,77.9,88.9,11.0,
A0307375,1,3A4,,30,49.0,57.4,8.4,
A0307377,1,3A4,,30,12.2,22.4,10.2,
A0307386,1,3A4,,30,27.4,55.4,28.0,
A0307387,1,3A4,,30,39.1,80.9,41.8,


# Export to excel files
- Check “Processed data” file, and if needed, amend cut-off value of %CV (variable “Val_pct_CV_cutoff”), and rerun the script;
- If you are happy with the results, move the two exported excel files and raw data file into your desired directory.

In [28]:
# Get the current date to attach to the file name
Val_current_date <- Sys.Date()                                                                                

List_Summary <- list("Summary" = Data_Summary, 
                     "3A4 (+)NADPH" = Df_CYP3A4_plus,
                     "3A4 (-)NADPH" = Df_CYP3A4_minus,
                     "2C8 (+)NADPH" = Df_CYP2C8_plus,
                     "2C8 (-)NADPH" = Df_CYP2C8_minus,
                     "2C9 (+)NADPH" = Df_CYP2C9_plus,
                     "2C9 (-)NADPH" = Df_CYP2C9_minus,
                     "2D6 (+)NADPH" = Df_CYP2D6_plus,
                     "2D6 (-)NADPH" = Df_CYP2D6_minus)

# Export data for summary to an excel file
write_xlsx(List_Summary,                                                                                      
           paste(Val_current_date, " CYP TDI 30min assay - 3A4 2C8 2C9 2D6 - Processed data", ".xlsx", sep = ""))  

# Export data for summary to an excel file
List_CDD_upload <- list("DatabaseResults" = CDD_upload_all)
write_xlsx(List_CDD_upload,                                                                                   
           paste(Val_current_date, " CYP TDI 30min assay - 3A4 2C8 2C9 2D6 - CDD upload", ".xlsx", sep = ""))  

# A little more work to do after running the script:
### Copy and paste process data summary into template, and add chemical structure
- Template file: R:\DMPK\1_In vitro\CYP450\TDI\2020-xx-xx CYP TDI 30min assay - 3A4 2C8 2C9 2D6 - Summary.xlsx.

### Open “CDD upload file” and check if anything needs to be deleted before upload;
- The script is designed to bind data of all compounds in compounds list except one positive control and DMSO.

### Send out the summary, upload on CDD, and you are all set!